# Thinking1 线性规划和混合整数规划的区别是什么？
            答：MIP(混合整数规划混合整数规划)要比LP(线性规划）的运算复杂度要高，运算花费的时间更长，因为单纯的线性规划，在计算最优解的时候更容易直接通过方程组计算的方式获得最终结果，算法上更容易优化然后更快的获得结果，但是MIP因为存在部分参数是限定为整数类型的，算法在计算的过程中需要暴力穷举可能的整数点，然后依次代入求解最优解，更加耗时。
# Thinking2 针对VRP,常见的约束条件有哪些？
            答：VRP求解的是多辆车同时去满足不同的用户，通常约束条件会有：1.需要覆盖每一个点或者每一条边；2.走最少的路径或者用最短的时间；3.用最少的成本等等。



# Action 1

In [1]:
import pandas as pd
data=pd.read_csv("G://python_lesson//L20//L22//santa//family_data.csv",index_col='family_id')
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [2]:
#preference cost 矩阵 pcost_mat
#accounting cost 矩阵 acost_mat
#家庭人数 FAMILY_SIZE
#每个家庭的倾向性选择（choice_）DESIRED
def get_penalty(n,choice):
    penalty=None
    if choice==0:
        penalty=0
    if choice==1:
        penalty=50
    if choice==2:
        penalty=50+9*n
    if choice==3:
        penalty=100+9*n
    if choice==4:
        penalty=200+9*n
    if choice==5:
        penalty=200+18*n
    if choice==6:
        penalty=300+18*n
    if choice==7:
        penalty=300+36*n
    if choice==8:
        penalty=400+36*n
    if choice==9:
        penalty=500+(36+199)*n
    if choice>9:
        penalty=500+(36+398)*n
    return penalty
        

In [3]:
import numpy as np
N_DAYS=100
N_FAMILY=5000
MIN_OCCUPANCY=125
MAX_OCCUPANCY=300

pcost_mat=np.full(shape=(N_FAMILY,N_DAYS),fill_value=999999)

for f in range(N_FAMILY):
    #f家庭的成员数
    f_num=data.loc[f,'n_people']
    #先把所有的数初始化为choice为10的penalty
    pcost_mat[f,:]=get_penalty(f_num,10)
    for choice in range(10):
        temp=data.loc[f][choice]
        penalty=get_penalty(f_num,choice)
        pcost_mat[f,temp-1]=penalty

pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [4]:
pcost_mat.shape

(5000, 100)

In [5]:
#accounting penalty 前一天和当天不同参观人数组合下的清洁费用
acost_mat=np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff=abs(i-j)
        acost_mat[i,j]=max(0,(i-125)/400*i**(0.5+diff/50.0))
acost_mat        
        


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [6]:
FAMILY_SIZE=data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [7]:
#每个家庭的期望访问日期
DESIRED=data.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

In [8]:
#线性规划
from ortools.linear_solver import pywraplp
def solveLP():
    solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x={}
    candidates=[[] for x in range(N_DAYS)]
    for i in range(N_FAMILY):
        for j in DESIRED[i,:]:
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]' %(i,j))
    #x[i,j]第i个家庭是否在第i天出现
    #daily_occupancy每天出现的总人数
    daily_occupancy=[solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]])\
                     for j in range(N_DAYS)]               
    family_prensence=[solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                      for i in range(N_FAMILY)]
    #目标函数最小值
    preference_cost=solver.Sum([pcost_mat[i,j]*x[i,j] for i in range(N_FAMILY) \
                                for j in DESIRED[i,:]])
    solver.Minimize(preference_cost)
    #任意两天出现的人数差值不要太大
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j]-daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1]-daily_occupancy[j]<=25)
    for i in range(N_FAMILY):
        solver.Add(family_prensence[i]==1)
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)            
    result=solver.Solve()
    temp=[(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    df=pd.DataFrame(temp,columns=['faminly_id','day','result'])
    return df
result=solveLP()
           

In [9]:
result

,faminly_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5068,4995,15,1.0
5069,4996,87,1.0
5070,4997,31,1.0
5071,4998,91,1.0


In [10]:
result['result'].value_counts()

1.000000    4301
1.000000     627
0.750000       2
0.666667       2
0.666667       2
            ... 
0.750000       1
0.400000       1
0.400000       1
0.400000       1
0.947917       1
Name: result, Length: 144, dtype: int64

In [11]:
#未安排的家庭
THRS=0.999
unassigned_df=result[(result.result<THRS)&(result.result>1-THRS)]
unassigned_df

,faminly_id,day,result
59,59,38,0.25
60,59,14,0.75
241,240,32,0.75
242,240,56,0.25
264,262,31,0.50
...,...,...,...
4983,4912,8,0.40
4985,4914,38,0.60
4986,4914,43,0.40
5033,4961,53,0.75


In [12]:
#计算每天到访总人数
assigened_df=result[result.result>THRS]
assigened_df['family_size']=FAMILY_SIZE[assigened_df.faminly_id]
assigened_df

<ipython-input-12-8cd08466a26a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigened_df['family_size']=FAMILY_SIZE[assigened_df.faminly_id]


,faminly_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5068,4995,15,1.0,4
5069,4996,87,1.0,2
5070,4997,31,1.0,6
5071,4998,91,1.0,5


In [13]:
occupancy=assigened_df.groupby('day').family_size.sum()
occupancy

day
0     290
1     271
2     294
3     293
4     263
     ... 
95    158
96    128
97    125
98    122
99    124
Name: family_size, Length: 100, dtype: int64

In [14]:
min_occupancy=np.array([max(0,MIN_OCCUPANCY-x)for x in occupancy])
max_occupancy=np.array([MAX_OCCUPANCY-x for x in occupancy])
print(min_occupancy)
print(max_occupancy)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 0 0 0 0 0 2 0 0 0 0
 0 0 4 0 0 0 0 0 0 5 0 0 0 0 0 0 0 3 0 0 0 0 0 0 3 1]
[ 10  29   6   7  37  58  77  53  27   3  12   8  25  50  62  28   8   8
  29  52  77  56  36   9   8   4  27  51  66  49  22  17  48  65  95 116
  98  67  47  69  90 117  96  71  53  19  44  77  96 102  78  52  45  77
  92 115 127 104  81 102 126 159 176 179 151 130 122 142 164 177 175 165
 142 115 142 165 179 175 158 133 114 133 162 180 172 145 126  97 123 148
 170 178 170 142 122 142 172 175 178 176]


In [15]:
#用整数规划求解剩余未安排的家庭
def solveIP(families,min_occupancy,max_occupancy):
    solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    n_families=len(families)
    
    x={}
    candidates=[[] for x in range(N_DAYS)]
    for i in families:
        for j in DESIRED[i,:]:
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]' %(i,j))
    #x[i,j]第i个家庭是否在第i天出现
    #daily_occupancy每天出现的总人数
    daily_occupancy=[solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]])\
                     for j in range(N_DAYS)]               
    family_prensence=[solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                      for i in families]
    #目标函数最小值
    preference_cost=solver.Sum([pcost_mat[i,j]*x[i,j] for i in families \
                                for j in DESIRED[i,:]])
    solver.Minimize(preference_cost)

    for i in range(n_families):
        solver.Add(family_prensence[i]==1)
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=max_occupancy[j])            
    result=solver.Solve()
    
    temp=[(i,j,x[i,j].solution_value()) for i in families for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    df=pd.DataFrame(temp,columns=['faminly_id','day','result'])
    return df
    


In [16]:
reslut1=solveIP(unassigned_df.faminly_id.unique(),min_occupancy,max_occupancy)
reslut1

,faminly_id,day,result
0,59,38,1.0
1,240,32,1.0
2,262,31,1.0
3,357,24,1.0
4,488,39,1.0
...,...,...,...
64,4869,59,1.0
65,4886,98,1.0
66,4912,17,1.0
67,4914,38,1.0


In [17]:
df=pd.concat((assigened_df[['faminly_id','day']],reslut1[['faminly_id','day']])).sort_values('faminly_id')
df

,faminly_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5068,4995,15
5069,4996,87
5070,4997,31
5071,4998,91


In [28]:
from numba import njit
#计算当前安排的成本
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy=np.zeros(N_DAYS+1,dtype=np.int64)
    penalty=0
    for (i,p) in enumerate(prediction):
        n=FAMILY_SIZE[i]
        penalty+=pcost_mat[i,p]
        daily_occupancy[p]+=n
    return penalty,daily_occupancy
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost=0
    num_out_range=0
    for day in range(N_DAYS):
        n_p1=daily_occupancy[day+1]
        n=daily_occupancy[day]
        num_out_range+=(n>MAX_OCCUPANCY)or(n<MIN_OCCUPANCY)
        accounting_cost+=acost_mat[n,n_p1]
    return accounting_cost,num_out_range
@njit(fastmath=True)
def cost_function(prediction):
    penalty,daily_occupancy=pcost(prediction)
    accounting_cost,num_out_range=acost(daily_occupancy)
    final_score=penalty+accounting_cost+num_out_range*999999999
    return final_score


In [29]:
prediction=df.day.values
final_cost=cost_function(prediction)
final_cost

109212.10210277568

In [30]:
final_cost

109212.10210277568

In [31]:
prediction=df.day.values
prediction

array([51, 25, 99, ..., 31, 91, 12], dtype=int64)

In [32]:
def save_result(pred,filename):
    result=pd.DataFrame(range(N_FAMILY),columns=['faminly_id'])
    result.to_csv(filename,index=False)
    print(filename+' saved')
    return result
result=save_result(prediction,'submission.csv')

submission.csv saved


In [33]:
#打雷法，遍历每个家庭的各项选择，并替代现有的安排列表，然后计算更新之后的cost是否更小，如果更小则替换
def find_better(pred):
    fobs=np.argsort(FAMILY_SIZE)
    score=cost_function(pred)
    original_score=np.inf
    while score<original_score:
        original_score=score
        for faminly_id in fobs:
            for pick in range(10):
                day=DESIRED[faminly_id,pick]
                oldvalue=pred[faminly_id]
                pred[faminly_id]=day
                new_socre=cost_function(pred)
                if new_socre<score:
                    score=new_socre
                else:
                    pred[faminly_id]=oldvalue
        print(score,end='\r')
        
        
        

In [34]:
new=prediction.copy()
find_better(new)


In [25]:
prediction

array([51, 25, 99, ..., 31, 91, 12], dtype=int64)

In [ ]:
cost_function(prediction)

# Acrion 2

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

In [2]:
df=pd.read_excel('G://python_lesson//L21//L21//cities.xlsx')
city_list=df["name"]
city_list

0       北京
1       天津
2      哈尔滨
3       长春
4       沈阳
5      石家庄
6     呼和浩特
7       太原
8       上海
9       杭州
10      福州
11      广州
12      海口
13      合肥
14      郑州
15      南京
16      济南
17      兰州
18      银川
19      西宁
20      西安
21      拉萨
22    乌鲁木齐
23      成都
24      重庆
25      南宁
26      贵阳
27      昆明
28      武汉
29      长沙
30      南昌
31      香港
32      澳门
Name: name, dtype: object

In [3]:
df1=pd.read_excel('G://python_lesson//L21//L21//distance.xlsx')
df1

,Unnamed: 0,北京,天津,哈尔滨,长春,沈阳,石家庄,呼和浩特,太原,上海,...,成都,重庆,南宁,贵阳,昆明,武汉,长沙,南昌,香港,澳门
0,北京,0,122476,1229869,987941,682963,294125,491658,487353,1210971,...,1835076,1753571,2346302,2086066,2587861,1161356,1477547,1431143,2203897,2273144
1,天津,122476,0,1201742,959814,654836,314982,614846,508210,1093989,...,1855933,1774428,2343851,2083615,2585410,1158905,1475096,1358857,2201446,2270693
2,哈尔滨,1229869,1201742,0,244488,551085,1493810,1684809,1687038,2269863,...,3034761,2953256,3533141,3272905,3774700,2348195,2664386,2534550,3390736,3459983
3,长春,987941,959814,244488,0,310181,1252906,1431315,1446134,2028959,...,2793857,2712352,3292237,3032001,3533796,2107291,2423482,2293646,3149832,3219079
4,沈阳,682963,654836,551085,310181,0,950093,1171538,1143321,1726146,...,2491044,2409539,2989424,2729188,3230983,1804478,2120669,1990833,2847019,2916266
5,石家庄,294125,314982,1493810,1252906,950093,0,594851,214615,1170170,...,1551896,1470391,2086876,1826211,2260044,909276,1218121,1243653,1974526,2013718
6,呼和浩特,491658,614846,1684809,1431315,1171538,594851,0,443246,1703117,...,1728512,1664862,2529565,2046276,2436660,1393252,1681446,1738006,2458502,2463568
7,太原,487353,508210,1687038,1446134,1143321,214615,443246,0,1365196,...,1371477,1289972,2094228,1671386,2079625,957915,1246109,1302669,2023165,2028231
8,上海,1210971,1093989,2269863,2028959,1726146,1170170,1703117,1365196,0,...,1956341,1702128,1904381,1816148,2317943,835747,1081808,700706,1493623,1563775
9,杭州,1277042,1217026,2392900,2151996,1849183,1180721,1724699,1357544,179942,...,1885005,1621852,1720463,1669561,2183377,755471,889395,515972,1309705,1379857


In [4]:
class tsp(object):
    def __init__(self,city_names=None,num_vehicles=1):
        self.df=pd.read_excel('G://python_lesson//L21//L21//cities.xlsx')
        self.all_city=self.df['name'].values
        self.num_vehicles=num_vehicles
        if city_names is not None:
            self.city_names=city_names
            self.df=self.df[self.df['name'].isin(city_names)]
        else:
            self.city_names=self.all_city
    
    # 设置数据,设置距离矩阵，设置车辆数
    def create_data_model(self):
        data = {}
        temp=pd.read_excel('G://python_lesson//L21//L21//distance.xlsx',index_col=0)
        temp=temp[temp.index.isin(self.city_names)][self.city_names]
        #print(temp)
        data['distance_matrix']=temp.values/1000
       # print(data)
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = 0
        return data
    def get_solution(self,manager, routing, solution):
        #print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        distance_list=[]
        route_list=[]
        for vehicle_id in range(self.num_vehicles):   
            route_distance = 0
            route=[]
            index = routing.Start(vehicle_id)#参数是车id
            #plan_output = '车辆的路径:\n'
            while not routing.IsEnd(index):
                #plan_output += ' {} ->'.format(city_names[manager.IndexToNode(index)])
                index_show=manager.IndexToNode(index)
                route.append(index_show)
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_list.append(route)
            distance_list.append(route_distance)
        return route_list,distance_list
      #  plan_output += city_names[manager.IndexToNode(index)]
       # print(plan_output)
        #print('{}公里'.format(route_distance))
    def add_distance_dimension(self,routing,transit_callback_index):
        dimension_name='Distance'
        routing.AddDimension(
            transit_callback_index,
            0,
            10000,
            True,
            dimension_name)
        distance_dimension=routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    def work(self):
     # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 计算两点之间的距离
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        self.add_distance_dimension(routing,transit_callback_index)

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        print('solution',solution)
        
        route,route_distance=self.get_solution(manager, routing, solution)
        return route,route_distance

if __name__=='__main__':
    #city_names=['北京','天津','上海']
    model=tsp(num_vehicles=4)
    model.create_data_model()
    
    #model=tsp()
    route,route_distance=model.work()
    print(route)
    print(route_distance)
    

solution Assignment(Distance0 (0) | Distance1 (4409) | Distance2 (5614) | Distance3 (5858) | Distance4 (5063) | Distance5 (294) | Distance6 (5850) | Distance7 (487) | Distance8 (3174) | Distance9 (2995) | Distance10 (2372) | Distance11 (5081) | Distance12 (4325) | Distance13 (5693) | Distance14 (690) | Distance15 (3474) | Distance16 (6340) | Distance17 (1635) | Distance18 (1208) | Distance19 (1208) | Distance20 (1085) | Distance21 (3548) | Distance22 (3550) | Distance23 (1860) | Distance24 (2163) | Distance25 (3840) | Distance26 (2546) | Distance27 (3065) | Distance28 (1202) | Distance29 (1527) | Distance30 (5249) | Distance31 (5249) | Distance32 (4921) | Distance33 (0) | Distance34 (0) | Distance35 (0) | Distance36 (6341) | Distance37 (7096) | Distance38 (6749) | Distance39 (6845) | Nexts0 (7) | Nexts1 (4) | Nexts2 (3) | Nexts3 (39) | Nexts4 (2) | Nexts5 (20) | Nexts6 (36) | Nexts7 (18) | Nexts8 (15) | Nexts9 (8) | Nexts10 (9) | Nexts11 (31) | Nexts12 (32) | Nexts13 (16) | Nexts14 (28